# Predictive Models for GHG Scope 1

In this notebook, we present the four predictive models we worked on.

## Feauture Engineering

Before coding the models, we will create two new variables in our dataset. 

1. Missing_GHG - Boolean indicating if the stock had a missing value in that year (1 = True / 0 = False)
2. Utilities - Boolean indicating if it is a Utility company (1 = True / 0 = False)
3. Time Trend - Cumulative value (0,1,2,3) for each year the stock is in the dataset.

The second variable will be used in the Model #3 and #4

Also, we will create a function to help us apply the k-fold cross validations to any number of splits and to whichever model.

#### Missing_GHG

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.feature_selection import RFE
from datetime import datetime, date
from mpl_toolkits.mplot3d import Axes3D # Visualize the Data for Multiple Linear Regression
import warnings
warnings.filterwarnings('ignore')

stocks = pd.read_csv("/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/pilot_stocks.csv")
sectors = pd.read_csv("/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/52_tickers_sectors.csv")

stocks['Missing_GHG'] = np.where(stocks['GHG Scope 1'].isna(), 1, 0)
stocks.loc[stocks['GHG Scope 1'].isna(),['GHG Scope 1','Missing_GHG']].head()

#### Utilities

In [ ]:
df = pd.merge(stocks, sectors, how='inner',on='Ticker')
df.drop(columns='Name', inplace=True)
stocks = df.copy()
stocks['Utility'] = np.where(stocks.Sector == 'Utilities',1,0)
stocks.loc[stocks.Sector == 'Utilities',].head()

#### Time trend

In [ ]:
stocks['time_trend'] = stocks.groupby('Ticker').cumcount()
stocks.loc[stocks.Ticker == 'XOM'].groupby(['Ticker','Year']).head()

### K-fold cross validation function

Below, we created a function called 'kfold_cross_validation' where we send the number of splits, X and y values and the model_type. The function creates the number of k-folds and fits and make predictions for each split. 


At the end, the function returns the Mean Squared error, the Root Mean Square Error and the Coefficient of determination (R2).

In [ ]:
def kfold_cross_validation(n_splits, X, y, model_type):
    data_y, data_yhat,coef = [], [],[]
    kfold = KFold(n_splits=n_splits, random_state = 42, shuffle=True)
    for train_ix, test_ix in kfold.split(X):
        # get data
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        # fit model
        model = model_type
        model.fit(train_X, train_y)
        # make predictions
        yhat = model.predict(test_X)
            
        coef.append(model.coef_)
        # store
        data_y.extend(test_y)
        data_yhat.extend(yhat)

    # Evaluate the model
    print('Mean squared error: %.2f' % metrics.mean_squared_error(data_y, data_yhat))
    print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(data_y, data_yhat))}')
    print('Coefficient of determination: %.2f'% metrics.r2_score(data_y, data_yhat))

## Models

### Model 1 - Sales and Assets


GHG = a + b1*Sales + b2*Assets + e     


In [ ]:
df=stocks.copy()
df.columns

For this model, the predictors we are interested in are Total Assets and Total Sales and the outcome variable is GHG Scope 1. We will proceed to subset the dataframe to get these columns. 

In [ ]:
da=df[['GHG Scope 1','Total_Sales', 'Total_Assets']]

We are now interested in the observations that are not missing GHG Scope. As presented in the previous notebooks, we have a lot of observations missing the GHG Scope 1 value. Therefore, from 780 observations we only get 398 observations. 

In [ ]:
dn=da.dropna()
print(f'Now, we have {dn.shape[0]} observations')

### Validation set approach - Hold out

Now, we create the X and y variables for the predictors and outcome variable, respectively. Also, we split the dataset into train (80%) and test (20%). 

In [ ]:
X = dn.iloc[:, 1:3].values
y = dn.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
print(f'When we do not include any explanatory variables, the GHG Scope 1 is {round(regressor.intercept_,3)}')

Now, we are going to predict the GHG Scope using the test split. 

In [ ]:
y_train_pred = regressor.predict(X_train)
y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.head()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
print('MSE train: %.3f, test: %.3f' % (metrics.mean_squared_error(y_train, y_train_pred),
                metrics.mean_squared_error(y_test, y_pred)))

The model is overfitting, it performs slightly better in train but poorly in test. We can apply other ML techniques to work with the overfitting and play with the Bias-Variance tradeoff in order to get a better accuracy.

In [ ]:
r2 =   metrics.r2_score(y_test, y_pred)
print('R^2: {0}'.format(r2))

As presented, the MSE is high which means our model isn't great and is overfitting. Moreover, this model explains around 55% of the variance in our dataset. Next, we will create a 3D visualization to present the predictions for this multiple linear regression. 

In [ ]:
df = pd.DataFrame(X_train ,columns=['Sales','Assets'])
df['GHG_Scope1']= pd.Series(y_train)

x_surf, y_surf = np.meshgrid(np.linspace(df.Assets.min(), df.Assets.max(), 100),np.linspace(df.Sales.min(), df.Sales.max(), 100))

onlyX = pd.DataFrame({'Sales': x_surf.ravel(), 'Assets': y_surf.ravel()})
fittedY= regressor.predict(onlyX)
fittedY=np.array(fittedY)

fig = plt.figure(figsize=(25,12))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df.Assets,df.Sales,df.GHG_Scope1,c='red', marker='o', alpha=0.5)
ax.plot_surface(x_surf,y_surf,fittedY.reshape(x_surf.shape), color='b', alpha=0.3)
ax.set_xlabel('Total Sales')
ax.set_ylabel('Total Assets')
ax.set_zlabel('GHG Scope 1')
fig.suptitle('Linear Regression - Sales and Assets vs. GHG Scope 1', fontsize=20);

Also, we can visualize the observed vs predictions to get a better idea of the residuals. 

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(y_test, y_pred, c='green')
ax.plot([y_test.min(), y_test.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Observed GHG Scope 1')
ax.set_ylabel('Predicted GHG Scope 1')
ax.set_title('Linear Regression- Hold out approach');

### K-fold cross validation

Now, we will apply another cross validation technique to improve the model predictions. We first did 10 k-folds and then 5 - k folds

In [ ]:
X = dn.iloc[:, 1:3].values
y = dn.iloc[:, 0].values
regressor = LinearRegression()
kfold_cross_validation(10,X,y,regressor)

In [ ]:
regressor = LinearRegression()
kfold_cross_validation(5,X,y,regressor)

10 k-fold cross validation presents a higher MSE than 5-fold cross validation. Both of them explained 46% of the variance which is lower than Linear regression - hold out approach. 

We will create a visualization to present the predictions for Linear Regression using 5-kfold cross validation.

In [ ]:
lr = LinearRegression()
X = dn.iloc[:, 1:3].values
y = dn.iloc[:, 0].values
predicted = cross_val_predict(lr, X, y, cv=5)

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(y, predicted, c='red')
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Observed GHG Scope 1')
ax.set_ylabel('Predicted GHG Scope 1')
ax.set_title('5-kfold cross-validation predictions');

As presented, the model does not fit the data well. The model accounts for 46% of the variance. Let's remember that the more variance that is accounted for by the regression model the closer the data points will fall to the fitted regression line. The visualization presents big residuals between the observed and the fitted GHG Scope 1. 

We will proceed to add this results to a dataframe where we can keep track of all the models and their respective MSE. 

In [ ]:
df_model_eval = pd.DataFrame({'Model_Number':1,'Model_Name': 'Linear (Sales + Asset)','Approach':'Hold-out','MSE': metrics.mean_squared_error(y_test, y_pred), 'RMSE' :np.sqrt(metrics.mean_squared_error(y_test, y_pred)), 'R-Squared' :  metrics.r2_score(y_test, y_pred)}, index=[0])
df_model_eval.loc[len(df_model_eval.index)] = [2, 'Linear (Sales + Asset)', '5 K-fold', 572243005.35, 23921.601228835698, 0.46] 
df_model_eval

### Ridge Regression 

#### Validation set approach (Hold-out)

We started with 100 alphas and instead of arbitrarily choosing alpha, we used cross-validation to choose the tuning parameter alpha. 

In [ ]:
n_alphas = 100
alphas = np.logspace(-10, -2, n_alphas)

# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)

ridge_1 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge_1.fit(X_train, y_train)
yhat_ridge = ridge_1.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_ridge))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_ridge))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_ridge))

#### K-fold cross validation


Now, we are going to use 5 and 10 k-fold cross validation to try to yield a lower MSE. We continued using the Ridge Regression with the best alpha but now using the K-fold cross validation to fit and predict. 

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X,y)
ridge_2 = Ridge(alpha = ridgecv.alpha_, normalize = True)
kfold_cross_validation(5,X,y,ridge_2)

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X,y)
ridge_3 = Ridge(alpha = ridgecv.alpha_, normalize = True)
kfold_cross_validation(10,X,y,ridge_3)

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [3, 'Ridge (Sales + Asset)', '5 K-fold', 574765481.16, 23974.267, 0.46] 
df_model_eval

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X,y)
ridge_3 = Ridge(alpha = ridgecv.alpha_, normalize = True)
predicted = cross_val_predict(ridge_3, X, y, cv=5)

fig, axs = plt.subplots(1,2, figsize=(25,10));
fig.suptitle('Ridge Regression - Total Assets and Total Sales vs. GHG Scope 1');
sns.scatterplot(x=y_test, y = yhat_ridge, ax=axs[0], color='b');
axs[0].set_title('Validation Set Approach - Hold out');
axs[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4);
sns.scatterplot(x=y, y = predicted, ax=axs[1], color='g');
axs[1].set_title('5-kfold cross validation');
axs[1].plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4);

Both models don't fit the data well. The model on the left accounts for 55% of the variance and the model on the right accounts for 46% of variance. Again, the more variance that is accounted for by the regression model the closer the data points will fall to the fitted regression line.

We can say that the model on the left is better but we need to take into consideration that it has less data points (Presenting Testing 20% of the data). On the other hand, it presents all the data points and the fitted line. It accounts for less variance but the RMSE decreases too. 

What do we prefer? Validation Set approach because it accounts for more variance and it will behave better with unseen data, right?

Why a flexible model isn't working that well?

### Lasso Regression

#### Validation Set approach - Hold out

First, we will plot the relationship between alpha and the weights (regression parameters), a line for each features. In this case, two (Sales and Assets). 

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []
alphas = 10**np.linspace(6,-2,100)*0.5

X = dn.iloc[:, 1:3].values
y = dn.iloc[:, 0].values

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X, y)
    coefs.append(lasso.coef_)

print(np.shape(coefs))
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

Now, we are going to subset into training and test and calculate the best optimal alpha using cross validation. (Tuning the hyperparameter)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

print(f'The optimal alpha is {lassocv.alpha_}')

In [ ]:
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
yhat_lasso = lasso.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_lasso))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_lasso))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_lasso ))

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [4, 'Lasso (Sales + Assets)', 'Hold-out',metrics.mean_squared_error(y_test,yhat_lasso), np.sqrt(metrics.mean_squared_error(y_test,yhat_lasso)), metrics.r2_score(y_test, yhat_lasso)] 
df_model_eval

#### K-fold Cross validation approach

Now, we are going to use k folds to fit and predict. We are going to continue using the best optimal alpha.

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
kfold_cross_validation(5, X, y, lasso)

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
kfold_cross_validation(10, X, y, lasso)

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
predicted = cross_val_predict(lasso, X, y, cv=5)

fig, axs = plt.subplots(1,2, figsize=(25,10));
fig.suptitle('Lasso Regression - Total Assets and Total Sales vs. GHG Scope 1');
sns.scatterplot(x=y_test, y = yhat_lasso, ax=axs[0], color='b');
axs[0].set_title('Validation Set Approach - Hold out');
axs[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4);
sns.scatterplot(x=y, y = predicted, ax=axs[1], color='g');
axs[1].set_title('5-kfold cross validation');
axs[1].plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4);

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [5, 'Lasso (Sales + Assets)', '5-kfold',572164056.06, 23919.95, 0.46] 
df_model_eval

### Model 2 - Logarithm Sales and Assets

### Linear Regression

#### Validation Set Approach - Hold out

ln(GHG) = a0 + b10*ln(Sales) + b20*ln(Assets) + e

In [ ]:
model_df = stocks[['Total_Sales', 'Total_Assets', 'GHG Scope 1']].copy()
model_df.dropna(inplace = True)

x = np.log(model_df[['Total_Sales', 'Total_Assets']])
y = np.log(model_df['GHG Scope 1'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
clf = LinearRegression().fit(X_train, y_train)

y_pred = np.exp(clf.predict(X_test))
y_test = np.exp(y_test)

In [ ]:
#The coefficients
print('Coefficients: \n',clf.coef_)
#Mean Squared Error
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test,y_pred))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test,y_pred))}')
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % metrics.r2_score(y_test, y_pred))

In [ ]:
log_reg = pd.DataFrame({'Observed':y_test,'Prediction':y_pred})
log_reg.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(log_reg.Observed, log_reg.Prediction, c='red')
ax.plot([log_reg.min(), log_reg.max()], [log_reg.min(), log_reg.max()], 'k--', lw=4)
ax.set_xlabel('Observed GHG Scope 1')
ax.set_ylabel('Predicted GHG Scope 1')
ax.set_title('Log(Total Assets) and Log (Total Sales) vs. GHG Scope 1 - Linear Regression Hold out');

This model accounts for 40% of the variance. Similar as the other regressions, as soon the GHG Scope starts to increase, the model starts making bad predictions (high bias, way out of target). 

We will proceed to add this results to the dataframe and continue to apply other ML techniques to try to yield a lower MSE. 

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [6, 'Linear (ln(Sales) + ln(Asset))', 'Hold-out',metrics.mean_squared_error(y_test,y_pred), np.sqrt(metrics.mean_squared_error(y_test,y_pred)), metrics.r2_score(y_test, y_pred)] 
df_model_eval

#### K-fold cross validation

Now, we will apply another cross validation technique to improve the model predictions. First, we will do 5 k-folds and then 10 - k folds. 

In [ ]:
df1 = stocks.loc[stocks['GHG Scope 1'].notna(),['GHG Scope 1','Logarithm_Total_Sales','Logarithm_Total_Assets']].copy()
X = df1.iloc[:,1:3].values
y = df1.iloc[:,0].values
regressor = LinearRegression()
kfold_cross_validation(5,X,y,regressor)

In [ ]:
X = df1.iloc[:,1:3].values
y = df1.iloc[:,0].values
regressor = LinearRegression()
kfold_cross_validation(10,X,y,regressor)

In [ ]:
lr = LinearRegression()
predicted = cross_val_predict(lr, X, y, cv=5)

df_kfold = pd.DataFrame({'Observed':y.flatten(), 'Predicted': predicted.flatten()})
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(y, predicted, c='red')
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Observed GHG Scope 1')
ax.set_ylabel('Predicted GHG Scope 1')
ax.set_title('5-kfold cross-validation predictions');

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [7 , 'Linear (ln(Sales) + ln(Asset))', '5-kfold',676789732.54,26015.18, 0.36] 
df_model_eval

### Ridge Regression 

#### Validation Set Approach - Hold out approach

Similar as model 1, we will continue applying different ML techniques to yield the lower MSE. First, we will go with Ridge Regression and apply the Hold Out approach which means we will use 80% for training and 20% for testing. 


We started with 100 alphas and instead of arbitrarily choosing alpha, we will use cross-validation to choose the tuning parameter alpha. 

In [ ]:
n_alphas = 100
alphas = np.logspace(-10, -2, n_alphas)

X = df1.iloc[:,1:3].values
y = df1.iloc[:,0].values

# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)

ridge_1 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge_1.fit(X_train, y_train)
yhat_ridge = ridge_1.predict(X_test)

# Evaluate the model
print('Coefficients:', ridge_1.coef_)
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_ridge))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_ridge))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_ridge))

#### K-fold validation

We applied cross validation to get the best alpha. However, now we are going to do k-fold cross validation to apply to train and test the data in different folds. 

We continued using the Ridge Regression with the best alpha but now using the K-fold cross validation to fit and predict. 

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X,y)
ridge_2 = Ridge(alpha = ridgecv.alpha_, normalize = True)
kfold_cross_validation(5,X,y,ridge_2)

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X,y)
ridge_2 = Ridge(alpha = ridgecv.alpha_, normalize = True)
kfold_cross_validation(10,X,y,ridge_2)

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X,y)
ridge_3 = Ridge(alpha = ridgecv.alpha_, normalize = True)
predicted = cross_val_predict(ridge_3, X, y, cv=5)

fig, axs = plt.subplots(1,2, figsize=(25,10));
fig.suptitle('Ridge Regression - ln(Total Sales) and ln(Total Assets) vs. GHG Scope 1');
sns.scatterplot(x=y_test, y = yhat_ridge, ax=axs[0], color='b');
axs[0].set_title('Validation Set Approach - Hold out');
axs[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4);
sns.scatterplot(x=y, y = predicted, ax=axs[1], color='g');
axs[1].set_title('5-kfold cross validation');
axs[1].plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4);

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [8 , 'Ridge (ln(Sales) + ln(Asset))', 'Hold-out',metrics.mean_squared_error(y_test,yhat_ridge), np.sqrt(metrics.mean_squared_error(y_test,yhat_ridge)), metrics.r2_score(y_test, yhat_ridge)] 
df_model_eval.loc[len(df_model_eval.index)] = [9 , 'Ridge (ln(Sales) + ln(Asset))', '5-kfolds',676843560.22, 26016.21, 0.36] 
df_model_eval

### Lasso Regression

#### Validation Set Approach

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []
alphas = 10**np.linspace(6,-2,100)*0.5

X = df1.iloc[:,1:3].values
y = df1.iloc[:,0].values

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X, y)
    coefs.append(lasso.coef_)

ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights');

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

print(f'The optimal alpha is {lassocv.alpha_}')

In [ ]:
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
yhat_lasso = lasso.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_lasso))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_lasso))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_lasso ))

#### K-fold validation

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
kfold_cross_validation(5, X, y, lasso)

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
kfold_cross_validation(10, X, y, lasso)

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
predicted = cross_val_predict(lasso, X, y, cv=5)

fig, axs = plt.subplots(1,2, figsize=(25,10));
fig.suptitle('Lasso Regression - ln(Total Sales) and ln(Total Assets) vs. GHG Scope 1');
sns.scatterplot(x=y_test, y = yhat_lasso, ax=axs[0], color='b');
axs[0].set_title('Validation Set Approach - Hold out');
axs[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4);
sns.scatterplot(x=y, y = predicted, ax=axs[1], color='g');
axs[1].set_title('5-kfold cross validation');
axs[1].plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4);

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [10 , 'Lasso (ln(Sales) + ln(Asset))', 'Hold-out',metrics.mean_squared_error(y_test,yhat_ridge), np.sqrt(metrics.mean_squared_error(y_test,yhat_ridge)), metrics.r2_score(y_test, yhat_ridge)] 
df_model_eval.loc[len(df_model_eval.index)] = [11 , 'Lasso (ln(Sales) + ln(Asset))', '5-kfolds',677276647.87, 26024.53, 0.36] 
df_model_eval

### Model 3 -  GHG for Energy vs Utilities

GHG = a + b1*Sales + b2*Assets + b3*Util + b5*TimeTrend + e

#### Linear Regression - Validation set approach

In [ ]:
stock = stocks.copy()
stock.dropna(inplace=True, subset = ['Total_Assets','Total_Sales','GHG Scope 1'])

X = stock[['Total_Sales','Total_Assets','Utility','time_trend']]
y = stock['GHG Scope 1']
print(X.shape)
print(y.shape)

In [ ]:
model = LinearRegression(fit_intercept=True) #Initialize model
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42) #Train and Test split
model.fit(Xtrain,ytrain) #Fit the model
y_model = model.predict(Xtest) #Make predictions

In [ ]:
#The coefficients
print('Coefficients: \n', model.coef_)
#Mean Squared Error
print('Mean squared error: %.2f' % metrics.mean_squared_error(ytest,y_model))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(ytest,y_model))}')
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % metrics.r2_score(ytest, y_model))

In [ ]:
#Create data frame with observed and predicted
linear_reg = pd.DataFrame({'Observed':ytest,'Prediction':y_model})
linear_reg.head()

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [12, 'Linear Regression (Sales + Asset + Utilities + Timetrend)', 'Hold-out', metrics.mean_squared_error(ytest,y_model), np.sqrt(metrics.mean_squared_error(ytest,y_model)), metrics.r2_score(ytest, y_model)]
df_model_eval

In [ ]:
# df = pd.DataFrame(X_train ,columns=['Sales','Assets','Utilities','Timetrend'])
# df['GHG_Scope1']= pd.Series(y_train)

# x_surf, y_surf, z_surf, zz_surf = np.meshgrid(np.linspace(df.Sales.min(), df.Sales.max(), 100),
#                                      np.linspace(df.Assets.min(), df.Assets.max(), 100),
#                                     np.linspace(df.Utilities.min(), df.Utilities.max(), 100),
#                                     np.linspace(df.Timetrend.min(), df.Timetrend.max(), 100))

# onlyX = pd.DataFrame({'Sales': x_surf.ravel(), 'Assets': y_surf.ravel(), 
#                       'Utilities': z_surf.ravel(), 'Timetrend': zz_surf.ravel()})
# fittedY= regressor.predict(onlyX)
# fittedY=np.array(fittedY)

# fig = plt.figure(figsize=(25,12))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(df.Sales,df.Assets,df.Utilities, df.Timetrend, df.GHG_Scope1,c='red', marker='o', alpha=0.5)
# ax.plot_surface(x_surf,y_surf,z_surf, zz_surf, fittedY.reshape(x_surf.shape), color='b', alpha=0.3)
# ax.set_xlabel('Total Sales')
# ax.set_ylabel('Total Assets')
# ax.set_zlabel('GHG Scope 1')
# fig.suptitle('Linear Regression - Sales and Assets vs. GHG Scope 1', fontsize=20);

#### Linear regression with k-fold cross validation

In [ ]:
X = X.values
y = y.values

In [ ]:
model1 = LinearRegression()
kfold_cross_validation(10,X,y,model1)

In [ ]:
kfold_cross_validation(5,X,y,model1)

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = [4, 'Linear Regression CV 5K-fold (Sales + Asset + Utilities + Timetrend)', 458669164.20,  21416.56 ,0.57 ] 

#### Ridge Regression - Validation set approach

In [ ]:
n_alphas = 100
alphas = np.logspace(-10, -2, n_alphas)

coefs = []
for a in alphas:
    ridge = Ridge(alpha=a, fit_intercept=False, normalize=True)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
    ridge.fit(Xtrain, ytrain)
    coefs.append(ridge.coef_)

ax = plt.gca()
ax.plot(alphas, coefs)
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.show()

Do we need to standarized the data? One feature is staying high.. would it look better if we normalize it? Would the logarithm work better because it distributes the data better?

In [ ]:
coefs[:5] #First five coefficients
#print(f'Last 5 coefficients: {coefs[(len(coefs) - 5):len(coefs)]}')

In [ ]:
coefs[(len(coefs) - 5):len(coefs)] #the last 5 coefficients

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Instead of arbitrarily choosing alpha, we used cross-validation to choose the tuning parameter alpha. 

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)


In [ ]:
ridge_1 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge_1.fit(X_train, y_train)
yhat_ridge = ridge_1.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_ridge))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_ridge))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_ridge))

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Ridge (Sales + Asset + Utilities + Timetrend)', metrics.mean_squared_error(y_test, yhat_ridge), np.sqrt(metrics.mean_squared_error(y_test, yhat_ridge)), metrics.r2_score(y_test, yhat_ridge)] 

#### Ridge Regression - Cross validation approach

In [ ]:
kfold_cross_validation(10,X,y,ridge_1)

In [ ]:
kfold_cross_validation(5,X,y,ridge_1)

In [ ]:
kfold_cross_validation(2,X,y,ridge_1)

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Ridge CV 5K-fold (Sales + Asset + Utilities + Timetrend)', 458487743.03, 21412.326894264337 ,0.57 ] 

#### Lasso - Validation set approach

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(X_train), y_train)
    coefs.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

print(f'The optimal alpha is {lassocv.alpha_}')

In [ ]:
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
yhat_lasso = lasso.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_lasso))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_lasso))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_lasso ))

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Lasso (Sales + Asset + Utilities + Timetrend)', metrics.mean_squared_error(y_test, yhat_lasso), np.sqrt(metrics.mean_squared_error(y_test, yhat_lasso)),metrics.r2_score(y_test, yhat_lasso ) ] 

#### Lasso Regression - Cross validation approach

In [ ]:
kfold_cross_validation(10, X,y,lasso)

In [ ]:
kfold_cross_validation(5, X,y,lasso)

In [ ]:
df_model_eval

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Lasso CV 5K-fold (Sales + Asset + Utilities + Timetrend)', 458655027.23, 21416.23279729868 ,0.57 ] 

### Model 4 -  GHG for Energy vs Utilities using natural logarithms


ln(GHG) = a + b1*ln(Sales) + b2*ln(Assets) + b3*ln(Util) + b5*ln(TimeTrend) + e

In [ ]:
X=stock[['Logarithm_Total_Sales','Logarithm_Total_Assets','Utility','time_trend']]
y=stock['GHG Scope 1']

In [ ]:
model = LinearRegression(fit_intercept=True) #Initialize model
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42) #Train and Test split
model.fit(Xtrain,ytrain) #Fit the model
y_model = model.predict(Xtest) #Make predictions

In [ ]:
#The coefficients
print('Coefficients: \n', model.coef_)
#Mean Squared Error
print('Mean squared error: %.2f' % metrics.mean_squared_error(ytest,y_model))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(ytest,y_model))}')
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % metrics.r2_score(ytest, y_model))

In [ ]:
#Create data frame with observed and predicted
linear_reg4 = pd.DataFrame({'Observed':ytest,'Prediction':y_model})
linear_reg4.head()

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Logarithm (Sales + Asset + Utilities + Timetrend)', metrics.mean_squared_error(ytest, y_model),np.sqrt(metrics.mean_squared_error(ytest, y_model)), metrics.r2_score(ytest, y_model)] 

#### Linear regression with k-fold cross validation

In [ ]:
X = X.values
y = y.values

def kfold_cross_validation(n_splits, X, y, model_type):
    data_y, data_yhat,coef = [], [],[]
    kfold = KFold(n_splits=n_splits, random_state = 42, shuffle=True)
    for train_ix, test_ix in kfold.split(X):
        # get data
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        # fit model
        model = model_type
        model.fit(train_X, train_y)
        # make predictions
        yhat = model.predict(test_X)
        coef.append(model.coef_)
        # store
        data_y.extend(test_y)
        data_yhat.extend(yhat)

    # Evaluate the model
    print('Mean squared error: %.2f' % metrics.mean_squared_error(data_y, data_yhat))
    print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(data_y, data_yhat))}')
    print('Coefficient of determination: %.2f'% metrics.r2_score(data_y, data_yhat))

In [ ]:
model1 = LinearRegression()
kfold_cross_validation(10,X,y,model1)

In [ ]:
kfold_cross_validation(5,X,y,model1)

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Logarithm Linear 5-kfold (Sales + Asset + Utilities + Timetrend)', 569033682.28,23854.42689056714,0.46 ] 

#### Ridge Regression - Validation set approach

In [ ]:
n_alphas = 100
alphas = np.logspace(-10, -2, n_alphas)

coefs = []
for a in alphas:
    ridge = Ridge(alpha=a, fit_intercept=False, normalize=True)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
    ridge.fit(Xtrain, ytrain)
    coefs.append(ridge.coef_)

ax = plt.gca()
ax.plot(alphas, coefs)
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.show()

Do we need to standarized the data? One feature is staying high.. would it look better if we normalize it? Would the logarithm work better because it distributes the data better?

In [ ]:
coefs[:5] #First five coefficients
#print(f'Last 5 coefficients: {coefs[(len(coefs) - 5):len(coefs)]}')

In [ ]:
coefs[(len(coefs) - 5):len(coefs)] #the last 5 coefficients

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Instead of arbitrarily choosing alpha, we used cross-validation to choose the tuning parameter alpha. 

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)


In [ ]:
ridge_1 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge_1.fit(X_train, y_train)
yhat_ridge = ridge_1.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_ridge))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_ridge))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_ridge))

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Logarithm Ridge (Sales + Asset + Utilities + Timetrend)', metrics.mean_squared_error(y_test, yhat_ridge), np.sqrt(metrics.mean_squared_error(y_test, yhat_ridge)), metrics.r2_score(ytest, yhat_ridge)] 

In [ ]:
df_model_eval.reset_index(inplace = True, drop = True)

#### Ridge Regression - Cross validation approach

In [ ]:
kfold_cross_validation(10,X,y,ridge_1)

In [ ]:
kfold_cross_validation(5,X,y,ridge_1)

In [ ]:
kfold_cross_validation(2,X,y,ridge_1)

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Logarithm Ridge 5-kfold (Sales + Asset + Utilities + Timetrend)', 569122863.08,23856.29,0.46 ] 

#### Lasso - Validation set approach

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(X_train), y_train)
    coefs.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

print(f'The optimal alpha is {lassocv.alpha_}')

In [ ]:
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
yhat_lasso = lasso.predict(X_test)

# Evaluate the model
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, yhat_lasso))
print(f'Root Mean Squared Error: {np.sqrt(metrics.mean_squared_error(y_test, yhat_lasso))}')
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, yhat_lasso ))

#### Lasso Regression - Cross validation approach

In [ ]:
kfold_cross_validation(10, X,y,lasso)

In [ ]:
kfold_cross_validation(5, X,y,lasso)

In [ ]:
df_model_eval.loc[len(df_model_eval.index)] = ['Logarithm Lasso 5-kfold (Sales + Asset + Utilities + Timetrend)', 569052835.46,23854.83,0.46 ] 

### Conclusion - Model performance

All four predictive models had a big RSME when predicting the GHG Scope. Model 3 - had the lower RSME and the best R2 with 0.66.

However, we need to continue with other approach to work with the model performance or consider other ways to impute the missing values for GHG Scope 1



In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df_model_eval[['Model','RMSE','R-Squared']]

In [ ]:
sns.set(rc={'figure.figsize':(25,8.27)})
g = sns.barplot('Model', 'MSE', data = df_model_eval)
for item in g.get_xticklabels():
    item.set_rotation(10)
    
g.figure.savefig('Model MSEs.png', bbox_inches='tight');

In [ ]:
sns.set(rc={'figure.figsize':(25,8.27)})
g = sns.barplot('Model', 'R-Squared', data = df_model_eval)
for item in g.get_xticklabels():
    item.set_rotation(10)
    
g.figure.savefig('Model R-Squared.png', bbox_inches='tight')